In [45]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from lxml import objectify
from lxml import etree
import xml.dom.minidom

import sys
sys.path.append(r'C:\Users\User\Documents\Code\blimp')
sys.path.append(r'C:\Users\User\Documents\Code\blimp\sdk')

#import blimp modules
from prairie_interface import prairie_interface
from SLM_sdk import SLM_sdk

In [2]:
#output Hz of Naparm
global naparm_hz #pre class
naparm_hz = 1000

In [3]:
naparm_path = r'F:\Data\rlees\2018-12-21\NAPARM\2018-12-21_RL006_NAPARM_008'

In [4]:
for file in os.listdir(naparm_path):
    file_path = os.path.join(naparm_path, file)
    
    if file.endswith('SLMAllTrials.dat'):
        trigger_path = file_path
    elif file.endswith('.gpl'):
        gpl_path = file_path
    elif file.endswith('.xml'):
        xml_path = file_path
    elif file == 'PhaseMasks':
        mask_path = file_path
    

mask_files = []

for file in os.listdir(mask_path):
    file_path = os.path.join(mask_path, file)
    if file.endswith('.tif'):
        mask_files.append(file_path)
  

In [5]:
def trigger_times(dat_path):
    
    ''' function to detect edges signalling trigger onset in dat files
        input: dat file path
        output: times of trigger onsets in dat file (seconds)
        
    '''
    
    arr = np.fromfile(dat_path, dtype=float)
    
    #edge detect the SLM trigger dat file      
    edges = [i for i,j in enumerate(arr) if arr[i-1] == 0 and arr[i] == 5]

    # above list doesnt hold for triggers sent at sample 0
    if arr[0] == 5:
        edges[0] = 0
            
    times = np.array(edges) / naparm_hz
    
    return times
        

In [6]:
triggers = trigger_times(trigger_path)

In [7]:
pi = prairie_interface()

PV connection successful


In [8]:
pi.load_markpoints(gpl_path)
pi.load_markpoints(xml_path)

In [9]:
sdk = SLM_sdk()
sdk.SLM_connect()

SLM connection established


In [10]:
num_masks = len(mask_files)
num_stims = len(triggers)
n_trials = num_stims / num_masks
assert n_trials.is_integer() == True and n_trials > 0
n_trials = int(n_trials)

In [71]:
#extract requried information from xlm into python lists

xml = etree.parse(xml_path)

laser_powers = []
spiral_revolutions = []
durations = []
delays = []

for elem in xml.iter():

    if elem.tag == 'PVMarkPointElement':
        laser_powers.append(elem.attrib['UncagingLaserPower'])
      
    elif elem.tag == 'PVGalvoPointElement':
        durations.append(elem.attrib['Duration'])
        spiral_revolutions.append(elem.attrib['SpiralRevolutions'])
        delays.append(elem.attrib['InitialDelay'])


In [72]:
gpl = etree.parse(gpl_path)

Xs = []
Ys = []
is_spirals = []
spiral_sizes = []


for elem in gpl.iter():
    
    if elem.tag == 'PVGalvoPoint':
        Xs.append(elem.attrib['X'])
        Ys.append(elem.attrib['Y'])
        is_spirals.append(elem.attrib['IsSpiral'])
        spiral_sizes.append(elem.attrib['SpiralSize'])


In [ ]:
-MarkPoints (-mp) "X Position (% of image 0-1, where 0 is the left side)" "Y Position (% of image 0-1, where 0 is the top of the image)" "Duration (ms)" "Laser Name (as it appears in the UI)" "Laser Power (same range as UI controls, supports 2P laser calibration)" ["Is Spiral (true/false)" "Spiral Size (% of image 0-1 in the x dimension, spiral will be forced to be a circle)" "Spiral Revolutions"] ["Trigger (None/PFI0/PFI1/PFI8/TrigIn)" ["TriggerCount (only used when Trigger is passed as PFI8)"]] ["Delay (ms)" "repeat all parameters again to mark another location"]

In [97]:
X = 0.8
Y = 1
duration = 5
laser_name = 'Uncaging'
laser_power = 1
is_spiral = 'true'
spiral_size = 10
spiral_revolutions = 3
trigger = 'None'
delay = 10


mp_str = '-MarkPoints ' + str(X) + ' ' + str(Y) + ' ' + str(duration) + ' ' + laser_name + ' ' + str(laser_power) +  ' ' + is_spiral + ' ' + str(spiral_size) + ' ' + str(spiral_revolutions) + ' ' + trigger + ' ' + str(delay)
pi.pl.SendScriptCommands(mp_str)

True

In [90]:
mp_str

'-MarkPoints 10 1 5 Uncaging 1 true 10 3 None 10'

In [81]:
mp_str

'-MarkPoints X Position 1 Y Position 0.5 Duration 5 Laser Name Uncaging Laser Power 1 Is Spiral true Spiral Size 10 Spiral Revolutions 3 Trigger None 5'

In [21]:
sleep_times = np.diff(triggers)
xml_path







'F:\\Data\\rlees\\2018-12-21\\NAPARM\\2018-12-21_RL006_NAPARM_008\\2018-12-21_RL006_NAPARM_008.xml'

1

In [12]:
trial_count = 0
for trial in range(n_trials):
    for mask in mask_files:    
        sdk.load_mask(mask)
        time.sleep(sleep_times[trial_count])
    

C:\Users\User\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\User\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\User\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
C:\Users\User\Anaconda3\lib\site-packages\imageio\plugins\_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
